In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
data_train = pd.read_csv('../input/titanic/train.csv')
data_test = pd.read_csv('../input/titanic/test.csv')
data_gen = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
data_train.columns

In [ ]:
data_train = data_train.merge(data_gen, on='PassengerId', how = 'left')
data_train = data_train.drop('Survived_y', axis = 1)
data_train = data_train.drop('Name', axis = 1)
data_train

In [ ]:
data_test = data_test.drop('Name', axis = 1)

In [ ]:
data_train.Age.fillna(round(data_train.Age.mean()), inplace = True)
data_test.Age.fillna(round(data_train.Age.mean()), inplace = True)
data_train

In [ ]:
data_train = pd.concat(
    [
        data_train,
        pd.get_dummies(data_train["Embarked"], prefix="Embarked"),
        pd.get_dummies(data_train["Sex"], prefix="Sex"),
    ],
    axis=1,
)
data_test = pd.concat(
    [
        data_test,
        pd.get_dummies(data_test["Embarked"], prefix="Embarked"),
        pd.get_dummies(data_test["Sex"], prefix="Sex"),
    ],
    axis=1,
)
data_train

In [ ]:
data_train = data_train.drop(['Embarked','Embarked_C','Embarked_Q', 'Sex','Ticket','Sex_male', 'Cabin'], axis = 1)
data_test = data_test.drop(['Embarked','Embarked_C','Embarked_Q', 'Sex','Ticket','Sex_male', 'Cabin'], axis = 1)
data_train

In [ ]:
data_test.info()

In [ ]:
data_train.info()

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train.drop('Survived_x', axis=1), data_train.Survived_x, train_size = 0.8, random_state = 322)

In [ ]:
params = {'max_depth':7, #глубина деревьев
    'boosting_type': 'gbdt', #модель для бустинга, в данном случае - gradient bosting decision tree
    'objective': 'binary', #задача
    'metric': ['auc'], #метрика качества
    'learning_rate': 0.07, #
     'seed': 322}

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
gbm = lgb.train(params, lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train, lgb_eval],
    early_stopping_rounds=75)

In [ ]:
train_titanic_prediction = gbm.predict(data_test)

In [ ]:
ttp = []
for i in train_titanic_prediction:
    if i < 0.5:
        ttp.append(0)
    else:
        ttp.append(1)

In [ ]:
data_test['Survived'] = ttp
data_test = data_test.drop(['Pclass','Age','SibSp','Parch','Fare','Embarked_S','Sex_female'], axis = 1)
data_test.to_csv('./file1.csv')

Анекдот для тех, кто досмотрел решение до конца (и учит пендосский):
— Эй, зачем ты пьешь эту грязную воду с озера, со всей деревни говносток сюда выходит!
— Whаt did уou sау?
— Я говорю: двумя руками черпай!

И еще немного актуалочки: Сидят два программиста в кафе. Мимо идет шикарная телка. — Классные у нее properties, — говорит один. — Вчера проверял... Все read only, — с грустью отвечает другой.